# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정 

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [ ]:
import zipfile

In [ ]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
dataset_path  = '/content/drive/MyDrive/Datasets/'
# dataset_path = 'C:/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [ ]:
import os

os.mkdir('/content/drive/MyDrive/Datasets')

FileExistsError: ignored

In [ ]:
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_train')

FileExistsError: ignored

In [ ]:
# 압축 해제
data = zipfile.ZipFile(file_path)
data.extractall('/content/drive/MyDrive/Datasets/Car_Images_train')

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [ ]:
import os
# 각각 경로 지정
# val 과 test 부터 만들어주자
folder_path_1 = '/content/drive/MyDrive/Datasets/Car_Images_val'
folder_path_2 = '/content/drive/MyDrive/Datasets/Car_Images_test' 

# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()
os.mkdir(folder_path_2)


# validation 폴더 만들기
os.mkdir(folder_path_1)

In [ ]:
# train, test, val 폴더에 normal과 abnormal을 만들어주자

folder_path_test_norm = '/content/drive/MyDrive/Datasets/Car_Images_test/normal'
folder_path_test_abnorm = '/content/drive/MyDrive/Datasets/Car_Images_test/abnormal'

folder_path_val_norm = '/content/drive/MyDrive/Datasets/Car_Images_val/normal'
folder_path_val_abnorm = '/content/drive/MyDrive/Datasets/Car_Images_val/abnormal'

# 생성

# os.mkdir(folder_path_train_norm)
# os.mkdir(folder_path_train_abnorm)
os.mkdir(folder_path_test_norm)
os.mkdir(folder_path_test_abnorm)
os.mkdir(folder_path_val_norm)
os.mkdir(folder_path_val_abnorm)


## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [ ]:
import random, shutil

In [ ]:
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train/normal'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal'

In [ ]:
# 전체 이미지 갯수를 확인합니다.
len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path))

(302, 303)

In [ ]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(tr_n_path))*0.2), round(len(os.listdir(tr_ab_path))*0.2)]
print(te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
val_data_num = [ round((len(os.listdir(tr_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(tr_n_path))-te_data_num[1])*0.2) ]
print(val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(tr_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(tr_ab_path))- te_data_num[1] - val_data_num[1]]

# 다 나눠준 후 train 사이즈
print(train_data_num)

[60, 61]
[48, 48]
[194, 194]


#### 2) test 셋 추출

In [ ]:
# train_normal에 들어가있는 파일
source_list_norm = [f for f in os.listdir(tr_n_path) if os.path.isfile(os.path.join(tr_n_path, f))] # normal 안에 들어가 있는 파일명 리스트
source_list_abnorm = [f for f in os.listdir(tr_ab_path) if os.path.isfile(os.path.join(tr_ab_path, f))] # abnormal 안에 들어가 있는 파일명 리스트

In [ ]:
random.seed(2023)

train_files_norm = source_list_norm[te_data_num[0]:] # 위에서 알려준 normal의 train_data의 사이즈, [0]은 normal
test_files_norm = source_list_norm[:te_data_num[0]] # 위에서 알려준 test_data의 사이즈

train_files_abnorm = source_list_abnorm[te_data_num[1]:] # 위에서 알려준 abnormal의 train_data의 사이즈, [1]은 normal
test_files_abnorm = source_list_abnorm[:te_data_num[1]] # 위에서 알려준 test_data의 사이즈

In [ ]:
for images in test_files_norm:
    source_path = os.path.join(tr_n_path, images)
    destination_path = os.path.join(folder_path_test_norm, images)
    shutil.copy(source_path, destination_path)

# 지금까지 한게 딱 test의 normal만 옮긴 것이다 위 코드들을 함수로 만들어서 바꿔주자

In [ ]:
# test파일에, train abnormal의 20퍼를 저장

for images in test_files_abnorm:
    source_path = os.path.join(tr_ab_path, images)
    destination_path = os.path.join(folder_path_test_abnorm, images)
    shutil.copy(source_path, destination_path)

In [ ]:
# 추출 후 이미지 갯수 확인
len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_test/normal')), len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_test/abnormal')) # 개수 맞는 것 같소.



(60, 61)

#### 3) validation 셋 추출

In [ ]:
# train_normal에 들어가있는 파일
random.seed(2023)



In [ ]:
# test_files_norm에 있는 원소들을 제외시켜줘야 한다
# norm
# abnorm
source_list_norm = [item for item in source_list_norm if item not in test_files_norm]
source_list_abnorm = [item for item in source_list_abnorm if item not in test_files_abnorm]

In [ ]:
len(source_list_norm), len(source_list_abnorm) # train에서 test를 제해준 사이즈는 맞다

(242, 242)

In [ ]:
train_files_norm = source_list_norm[val_data_num[0]:] # 위에서 알려준 normal의 train_data의 사이즈, [0]은 normal
val_files_norm = source_list_norm[:val_data_num[0]] # 위에서 알려준 test_data의 사이즈

train_files_abnorm = source_list_abnorm[val_data_num[1]:] # 위에서 알려준 abnormal의 train_data의 사이즈, [1]은 normal
val_files_abnorm = source_list_abnorm[:val_data_num[1]] # 위에서 알려준 test_data의 사이즈

In [ ]:
# validation normal 뽑기
for images in val_files_norm:
    source_path = os.path.join(tr_n_path, images)
    destination_path = os.path.join(folder_path_val_norm, images)
    shutil.copy(source_path, destination_path)

In [ ]:
# validation abnormal 뽑기
for images in val_files_abnorm:
    source_path = os.path.join(tr_ab_path, images)
    destination_path = os.path.join(folder_path_val_abnorm, images)
    shutil.copy(source_path, destination_path)

In [ ]:
len(source_list_norm), len(source_list_abnorm)

(242, 242)

In [ ]:
# 추출 후 이미지 갯수 확인


len(os.listdir(folder_path_val_norm)), len(os.listdir(folder_path_val_abnorm))

(48, 48)

In [ ]:
len(os.listdir(tr_n_path)), len(os.listdir(tr_ab_path))

(302, 303)

In [ ]:
# 마지막으로 옮겨준 test_normal, test_abnormal, val_normal, val_abnormal을 train 파일에서 없애줘야 한다

# 1. test_normal

for images in test_files_norm:
    # images는 파일명이 들어가있다
    # file_path안에 tr_n_path속 images를 결합시켜 없애주도록하자
    file_path = os.path.join(tr_n_path, images)
    os.remove(file_path)

for images in test_files_abnorm:
    file_path = os.path.join(tr_ab_path, images)
    os.remove(file_path)

In [ ]:
# validation 없애기
    
for images in val_files_norm:
    file_path = os.path.join(tr_n_path, images)
    os.remove(file_path)

for images in val_files_abnorm:
    file_path = os.path.join(tr_ab_path, images)
    os.remove(file_path)

In [ ]:
len(os.listdir(tr_n_path)), len(os.listdir(tr_ab_path))

(194, 194)

### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

* 복사하기 : shutil.copytree()

In [ ]:
from distutils.dir_util import copy_tree

In [ ]:
# 
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal', '/content/drive/MyDrive/Datasets/copy_images/trainset')
copy_tree('/content/drive/MyDrive/Datasets/Car_Images_train/normal', '/content/drive/MyDrive/Datasets/copy_images/trainset')

shutil.copytree('/content/drive/MyDrive/Datasets/Car_Images_val/abnormal', '/content/drive/MyDrive/Datasets/copy_images/validset')
copy_tree('/content/drive/MyDrive/Datasets/Car_Images_val/normal', '/content/drive/MyDrive/Datasets/copy_images/validset')

shutil.copytree('/content/drive/MyDrive/Datasets/Car_Images_test/abnormal', '/content/drive/MyDrive/Datasets/copy_images/testset')
copy_tree('/content/drive/MyDrive/Datasets/Car_Images_test/normal', '/content/drive/MyDrive/Datasets/copy_images/testset')


* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [ ]:
file_dir_path = '/content/drive/MyDrive/Datasets/copy_images/trainset'
file_list = os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal')
# train set에서의 abnormal, 위치는 copy_images/trainset이다
for file in file_list:
    original_path = os.path.join(file_dir_path, file)
    new_name = 'ab_' + file
    new_path = os.path.join(file_dir_path, new_name)
    os.rename(original_path, new_path)

In [ ]:
file_dir_path = '/content/drive/MyDrive/Datasets/copy_images/validset'
file_list = os.listdir('/content/drive/MyDrive/Datasets/Car_Images_val/abnormal')
# valid set에서의 abnormal, 위치는 copy_images/validset이다
for file in file_list:
    original_path = os.path.join(file_dir_path, file)
    new_name = 'ab_' + file
    new_path = os.path.join(file_dir_path, new_name)
    os.rename(original_path, new_path)

In [ ]:
file_dir_path = '/content/drive/MyDrive/Datasets/copy_images/testset'
file_list = os.listdir('/content/drive/MyDrive/Datasets/Car_Images_test/abnormal')
# test set에서의 abnormal, 위치는 copy_images/testset이다
for file in file_list:
    original_path = os.path.join(file_dir_path, file)
    new_name = 'ab_' + file
    new_path = os.path.join(file_dir_path, new_name)
    os.rename(original_path, new_path)

#### 2) normal 파일 복사

* 데이터 갯수 조회

In [ ]:
print(len(os.listdir(dataset_path+'copy_images/trainset/')))
print(len(os.listdir(dataset_path+'copy_images/validset/')))
print(len(os.listdir(dataset_path+'copy_images/testset/')))

# 오늘까지 해야하는 작업.

388
96
121


## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [219]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [ ]:
# 이미지 목록 저장
img_train_list = os.listdir('/content/drive/MyDrive/Datasets/copy_images/trainset')
img_valid_list = os.listdir('/content/drive/MyDrive/Datasets/copy_images/validset')
img_test_list = os.listdir('/content/drive/MyDrive/Datasets/copy_images/testset')

In [ ]:
ab_train = [image for image in img_train_list if image.startswith('ab_')]
ab_val = [image for image in img_valid_list if image.startswith('ab_')]
ab_test = [image for image in img_test_list if image.startswith('ab_')]

norm_train = [image for image in img_train_list if image not in ab_train]
norm_val = [image for image in img_valid_list if image not in ab_val]
norm_test = [image for image in img_test_list if image not in ab_test]

In [ ]:
# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 280 ## 사이즈 조정 가능

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [ ]:
! pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
len(img_train_list)

388

In [ ]:
from PIL import Image

x_train = []
y_train = []

# train데이터의 abnormal 부터 담아주자

for file in img_train_list:
    # file은 img_train_list속 normal과 ab이 섞여있는 파일명의 리스트이다.
    file_path = os.path.join('/content/drive/MyDrive/Datasets/copy_images/trainset', file)
    if file.startswith('ab_'):
        img = Image.open(file_path)
        img = img.resize((img_size, img_size))
        img_array = np.array(img)
        x_train.append(img_array)
        y_train.append(0)

    else:
        img = Image.open(file_path)
        img = img.resize((img_size, img_size))
        img_array = np.array(img)
        x_train.append(img_array)
        y_train.append(1)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

((382, 280, 280, 3), (382,))

In [ ]:
x_val = []
y_val = []

# train데이터의 abnormal 부터 담아주자

for file in img_valid_list:
    # file은 img_train_list속 normal과 ab이 섞여있는 파일명의 리스트이다.
    file_path = os.path.join('/content/drive/MyDrive/Datasets/copy_images/validset', file)
    if file.startswith('ab_'):
        img = Image.open(file_path)
        img = img.resize((img_size, img_size))
        img_array = np.array(img)
        x_val.append(img_array)
        y_val.append(0)

    else:
        img = Image.open(file_path)
        img = img.resize((img_size, img_size))
        img_array = np.array(img)
        x_val.append(img_array)
        y_val.append(1)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
x_val.shape, y_val.shape

((96, 280, 280, 3), (96,))

In [ ]:
x_test = []
y_test = []

# train데이터의 abnormal 부터 담아주자

for file in img_test_list:
    # file은 img_train_list속 normal과 ab이 섞여있는 파일명의 리스트이다.
    file_path = os.path.join('/content/drive/MyDrive/Datasets/copy_images/testset', file)
    if file.startswith('ab_'):
        img = Image.open(file_path)
        img = img.resize((img_size, img_size))
        img_array = np.array(img)
        x_test.append(img_array)
        y_test.append(0)

    else:
        img = Image.open(file_path)
        img = img.resize((img_size, img_size))
        img_array = np.array(img)
        x_test.append(img_array)
        y_test.append(1)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
x_test.shape, y_test.shape

((121, 280, 280, 3), (121,))

In [ ]:
len(img_train_list)

382

### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [ ]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

388
194
---
96
48
---
121
61


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import *

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [328]:
es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 5,
                   verbose = 0,
                   restore_best_weights = True)

In [ ]:
keras.backend.clear_session()
#     0. Functional, Sequential 중 택일

#     1. 인풋레이어
il = keras.layers.Input(shape = (280,280, 3))
#     2. Convolution : 필터수 32개, 사이즈(3, 3), same padding
cl = keras.layers.Conv2D(filters = 32,
                         kernel_size = (3,3),
                         strides = (1,1),
                         padding = 'same',
                         activation = 'relu')(il)
#     3. Convolution : 필터수 32개, 사이즈(3, 3), same padding
cl = keras.layers.Conv2D(filters = 32,
                         kernel_size = (3,3),
                         padding = 'same',
                         activation = 'relu')(cl)
#     4. BatchNormalization
bl = keras.layers.BatchNormalization()(cl)
#     5. MaxPooling : 사이즈(2,2) 스트라이드(2,2)
ml = keras.layers.MaxPool2D(pool_size = (2,2))(bl)
#     6. DropOut : 25% 비활성화
dl = keras.layers.Dropout(rate = 0.25)(ml)
#     7. Convolution : 필터수 64개, 사이즈(3, 3), same padding
cl = keras.layers.Conv2D(filters = 64,
                         kernel_size = (3,3),
                         padding = 'same',
                         activation = 'relu')(dl)
#     8. Convolution : 필터수 64개, 사이즈(3, 3), same padding
cl = keras.layers.Conv2D(filters = 64,
                         kernel_size = (3,3),
                         padding = 'same',
                         activation = 'relu')(cl)
#     9. BatchNormalization
bl = keras.layers.BatchNormalization()(cl)
#     10. MaxPooling : 사이즈(2,2) 스트라이드(2,2)
ml = keras.layers.MaxPool2D(pool_size = (2,2))(bl)
#     11. DropOut : 25% 비활성화
dl = keras.layers.Dropout(0.25)(ml)
#     12. Flatten( )
fl = keras.layers.Flatten()(dl)
#     13. Fully Connected Layer : 노드 1024개
hl = keras.layers.Dense(1024, activation = 'relu')(fl)
#     14. BatchNormalization
bl = keras.layers.BatchNormalization()(hl)
#     15. DropOut : 35% 비활성화
dl = keras.layers.Dropout(0.35)(bl)
#     16. 아웃풋레이어
ol = keras.layers.Dense(1, activation = 'sigmoid')(dl)

model = keras.models.Model(il,ol)

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'accuracy')

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 280, 280, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 280, 280, 32)      9248      
                                                                 
 batch_normalization (BatchN  (None, 280, 280, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 140, 140, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 140, 140, 32)      0     

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [161]:
from sklearn.utils import shuffle
import random

In [162]:
rand_n = np.random.randint(0, 10000)
rand_n

8076

In [163]:
x_train = shuffle(x_train, random_state = rand_n)
y_train = shuffle(y_train, random_state = rand_n)

In [ ]:
y_train

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,

In [ ]:
model.fit(x_train, y_train, epochs = 10000, verbose = 1, callbacks = [es], validation_data = (x_val,y_val))

# accuracy와 val_accuracy가 유지 되는 것은 어마무시한 overfitting이 일어나고 있다고 한다.
# stackoverflow에서는 data augmentation이나 더 작은 network를 만들어보라고 말하고 있다.
# Data Agumentation 어케 하지?
# 위에 있는 랜덤 인자를 넣어줘서 좀 더 숫자를 무작위로 섞은 결과, 좀 보기 좋은 결과가 나왔다
# 하지만 accuracy는 이상하게 높아지고 있는데, val_accuracy가 높아지지 않는 것을 보면 똑같이 overfitting이 일어나고 있는 것 같다.

Epoch 1/10000
13/13 [==============================] - 32s 930ms/step - loss: 1.4343 - accuracy: 0.7912 - val_loss: 78.4681 - val_accuracy: 0.5000
Epoch 2/10000
13/13 [==============================] - 4s 332ms/step - loss: 0.7184 - accuracy: 0.8454 - val_loss: 102.9556 - val_accuracy: 0.5000
Epoch 3/10000
13/13 [==============================] - 10s 816ms/step - loss: 0.2208 - accuracy: 0.9253 - val_loss: 5.8234 - val_accuracy: 0.5000
Epoch 4/10000
13/13 [==============================] - 4s 343ms/step - loss: 0.1566 - accuracy: 0.9278 - val_loss: 6.7349 - val_accuracy: 0.5000
Epoch 5/10000
13/13 [==============================] - 7s 541ms/step - loss: 0.0842 - accuracy: 0.9768 - val_loss: 0.6846 - val_accuracy: 0.6771
Epoch 6/10000
13/13 [==============================] - 4s 337ms/step - loss: 0.0840 - accuracy: 0.9716 - val_loss: 1.5121 - val_accuracy: 0.5312
Epoch 7/10000
13/13 [==============================] - 4s 335ms/step - loss: 0.0365 - accuracy: 0.9948 - val_loss: 0.9981 - v

In [ ]:
y_pred = model.predict(x_test)

4/4 [==============================] - 1s 331ms/step


In [ ]:
y_pred[:5], y_test[:5]

(array([[0.01140201],
        [0.00194263],
        [0.02028764],
        [0.9969703 ],
        [0.7470803 ]], dtype=float32), array([0, 0, 0, 0, 0]))

In [ ]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [ ]:
f1_score(y_pred, y_test)

0.7605633802816902

### (4) 모델2
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.


- **1. overfitting을 줄이자 1번, 적은 데이터에 알맞은 model 사용**
    - 이 경우 EfficientNet을 사용해보도록 하자

#### 1) 구조 설계

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import efficientnet

# # 나는 EfficientNetB0을 pretrained model로 가져올게요
# base_model = efficientnet.EfficientNetB0(include_top = True, # 그림으로 보면 주로 아래로 쌓이는데 왜 include top일까?
#                                                               # tensorflow 공식문서 include_top에 대해서는 이렇게 설명 되어있다.
#                                                               # top에 fully connected layer가 있는 경우 
#                                          weights ='imagenet', 
#                                          input_shape = (224, 224, 3))

# # pretrained 된 모델 (output 빼고) freeze -> 가중치 가져오겠단 말
# for layer in base_model.layers:
#     layer.trainable = False

# # 시작과 끝 엮기
# # 여기서 뭣모르고 input을 il했다가는 pretrained 다 날리고 layer 층 5개만 쌓인다.
# model = models.Model(base_model.input, base_model.output)

# # Compile the model
# model.summary()

In [ ]:
# 나는 EfficientNetB0을 pretrained model로 가져올게요
base_model = efficientnet.EfficientNetB0(include_top = False, # 그림으로 보면 주로 아래로 쌓이는데 왜 include top일까? 
                                         weights ='imagenet', # imagenet 바탕으로 미리 학습된 것 (include_top을 True로 해줬을 때, 
                                                              # 마지막 레이어가 1000으로 나누어준다는 사실을 알 수 있다.)
                                         input_shape = (280, 280, 3))

# pretrained 된 모델 (output 빼고) freeze -> 가중치 가져오겠단 말
for layer in base_model.layers:
    layer.trainable = False

# Functional API를 이용한 엮기
il = base_model.output
cl = layers.GlobalAveragePooling2D()(il)
hl = layers.Dense(128, activation='relu')(cl)
hl = layers.Dropout(0.5)(hl)
ol = layers.Dense(1, activation='sigmoid')(hl)

# 시작과 끝 엮기
# 여기서 뭣모르고 input을 il했다가는 pretrained 다 날리고 layer 층 5개만 쌓인다.
# 반드시 inputs = base_model.input
model = models.Model(base_model.input, ol)

# 컴파일
model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

# 요약
model.summary()

16705208/16705208 [==============================] - 2s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 280, 280, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 280, 280, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 280, 280, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)   

In [ ]:
len(model.layers)

# 242개의 층으로 되어있다는 것을 확인할 수 있었다
# 결과가 좋지 않다면 마지막 한 30, 50개 정도의 layers의 가중치는 버리고 다시 해보자

242

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
model.fit(x_train, y_train, epochs=10000, validation_data=(x_val, y_val), callbacks = [es], batch_size = 64)

Epoch 1/10000
7/7 [==============================] - 14s 880ms/step - loss: 0.4545 - accuracy: 0.7990 - val_loss: 0.1562 - val_accuracy: 0.9583
Epoch 2/10000
7/7 [==============================] - 2s 225ms/step - loss: 0.1594 - accuracy: 0.9613 - val_loss: 0.0930 - val_accuracy: 0.9792
Epoch 3/10000
7/7 [==============================] - 2s 223ms/step - loss: 0.0864 - accuracy: 0.9794 - val_loss: 0.0558 - val_accuracy: 1.0000
Epoch 4/10000
7/7 [==============================] - 2s 244ms/step - loss: 0.0610 - accuracy: 0.9845 - val_loss: 0.0457 - val_accuracy: 1.0000
Epoch 5/10000
7/7 [==============================] - 2s 304ms/step - loss: 0.0460 - accuracy: 0.9897 - val_loss: 0.0357 - val_accuracy: 1.0000
Epoch 6/10000
7/7 [==============================] - 2s 325ms/step - loss: 0.0292 - accuracy: 0.9923 - val_loss: 0.0294 - val_accuracy: 1.0000
Epoch 7/10000
7/7 [==============================] - 1s 204ms/step - loss: 0.0283 - accuracy: 0.9948 - val_loss: 0.0391 - val_accuracy: 1.000

In [ ]:
y_pred = model.predict(x_test)

4/4 [==============================] - 3s 363ms/step


In [ ]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.80      0.88        61
           1       0.83      0.98      0.90        60

    accuracy                           0.89       121
   macro avg       0.91      0.89      0.89       121
weighted avg       0.91      0.89      0.89       121



#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
# data agumentation을 진행해주고 나서 해보면 어떨까? 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
len(x_train)

388

In [ ]:
file_path = ''
os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/normal')

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 180,
    zoom_range = 0.3,
    zca_whitening = False,
    horizontal_flip = True,
    vertical_flip = True,
    
)

datagen.fit(x_train)

In [ ]:
model.fit(datagen.flow(x_train, y_train), epochs = 1000, validation_data = (x_val, y_val), verbose = 1, callbacks = [es])

Epoch 1/1000
13/13 [==============================] - 12s 649ms/step - loss: 0.1127 - accuracy: 0.9510 - val_loss: 0.0494 - val_accuracy: 0.9896
Epoch 2/1000
13/13 [==============================] - 7s 508ms/step - loss: 0.0892 - accuracy: 0.9639 - val_loss: 0.1888 - val_accuracy: 0.9375
Epoch 3/1000
13/13 [==============================] - 8s 623ms/step - loss: 0.0944 - accuracy: 0.9665 - val_loss: 0.0907 - val_accuracy: 0.9688
Epoch 4/1000
13/13 [==============================] - 7s 523ms/step - loss: 0.0692 - accuracy: 0.9742 - val_loss: 0.0227 - val_accuracy: 0.9896
Epoch 5/1000
13/13 [==============================] - 8s 626ms/step - loss: 0.0588 - accuracy: 0.9768 - val_loss: 0.0264 - val_accuracy: 0.9896
Epoch 6/1000
13/13 [==============================] - 7s 509ms/step - loss: 0.0405 - accuracy: 0.9845 - val_loss: 0.0140 - val_accuracy: 1.0000
Epoch 7/1000
13/13 [==============================] - 8s 573ms/step - loss: 0.0395 - accuracy: 0.9871 - val_loss: 0.0160 - val_accuracy

In [ ]:
y_pred = model.predict(x_test)

4/4 [==============================] - 0s 81ms/step


In [ ]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [ ]:
print(classification_report(y_test, y_pred))
# data agumentation을 잘 해줬는지 모르겠을 뿐더라, 일단은 여기까지 인 것 같다.

              precision    recall  f1-score   support

           0       0.98      0.74      0.84        61
           1       0.79      0.98      0.87        60

    accuracy                           0.86       121
   macro avg       0.88      0.86      0.86       121
weighted avg       0.88      0.86      0.86       121



### (5) 모델3
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)

In [263]:
len(base_model.layers)

238

In [372]:
# 나는 EfficientNetB0을 pretrained model로 가져올게요
base_model = tf.keras.applications.VGG16(include_top = False, # 그림으로 보면 주로 아래로 쌓이는데 왜 include top일까? 
                                          weights ='imagenet', # imagenet 바탕으로 미리 학습된 것 (include_top을 True로 해줬을 때, 
                                          # 마지막 레이어가 1000으로 나누어준다는 사실을 알 수 있다.)
                                          input_shape = (280, 280, 3))

# pretrained 된 모델 (output 빼고) freeze -> 가중치 가져오겠단 말
for idx, layer in enumerate(base_model.layers) :
    if idx < 225 :
        layer.trainable = False # 가중치는 가지고
    else :
        layer.trainable = True

# Functional API를 이용한 엮기

il = base_model.output
cl = layers.GlobalAveragePooling2D()(il)
bl = layers.BatchNormalization()(cl)
hl = layers.Dense(256, activation='relu')(bl)
hl = layers.Dropout(0.3)(hl)
ol = layers.Dense(1, activation='sigmoid')(hl)

# 시작과 끝 엮기
# 여기서 뭣모르고 input을 il했다가는 pretrained 다 날리고 layer 층 5개만 쌓인다.
# 반드시 inputs = base_model.input
model = models.Model(base_model.input, ol)

# 컴파일
model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [367]:
y_pred = model.predict(x_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
model.fit(x_train, y_train, epochs=10000, validation_data=(x_val, y_val), callbacks = [es], batch_size = 32)
y_pred = model.predict(x_test)
y_pred = np.where(y_pred > 0.5, 1 ,0)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

4/4 [==============================] - 5s 561ms/step
Epoch 1/10000
12/12 [==============================] - 11s 598ms/step - loss: 0.6771 - accuracy: 0.7592 - val_loss: 1.8915 - val_accuracy: 0.6042
Epoch 2/10000
12/12 [==============================] - 4s 330ms/step - loss: 0.2689 - accuracy: 0.9084 - val_loss: 0.6679 - val_accuracy: 0.7708
Epoch 3/10000
12/12 [==============================] - 4s 320ms/step - loss: 0.1675 - accuracy: 0.9424 - val_loss: 0.7314 - val_accuracy: 0.7708
Epoch 4/10000
12/12 [==============================] - 4s 334ms/step - loss: 0.0899 - accuracy: 0.9791 - val_loss: 0.6022 - val_accuracy: 0.7812
Epoch 5/10000
12/12 [==============================] - 4s 337ms/step - loss: 0.0424 - accuracy: 0.9974 - val_loss: 0.5566 - val_accuracy: 0.8333
Epoch 6/10000
12/12 [==============================] - 4s 321ms/step - loss: 0.0357 - accuracy: 0.9948 - val_loss: 0.6741 - val_accuracy: 0.8021
Epoch 7/10000
12/12 [==============================] - 4s 313ms/step - loss:

In [368]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [369]:
model.fit(x_train, y_train, epochs=10000, validation_data=(x_val, y_val), callbacks = [es], batch_size = 32)

Epoch 1/10000
12/12 [==============================] - 4s 370ms/step - loss: 0.0372 - accuracy: 0.9974 - val_loss: 0.6161 - val_accuracy: 0.7917
Epoch 2/10000
12/12 [==============================] - 4s 301ms/step - loss: 0.0430 - accuracy: 0.9895 - val_loss: 0.5837 - val_accuracy: 0.7917
Epoch 3/10000
12/12 [==============================] - 4s 302ms/step - loss: 0.0355 - accuracy: 0.9948 - val_loss: 0.5916 - val_accuracy: 0.8021
Epoch 4/10000
12/12 [==============================] - 4s 313ms/step - loss: 0.0289 - accuracy: 0.9921 - val_loss: 0.5786 - val_accuracy: 0.8333
Epoch 5/10000
12/12 [==============================] - 4s 308ms/step - loss: 0.0246 - accuracy: 0.9948 - val_loss: 0.7043 - val_accuracy: 0.7917
Epoch 6/10000
12/12 [==============================] - 4s 312ms/step - loss: 0.0163 - accuracy: 0.9974 - val_loss: 0.6261 - val_accuracy: 0.8125
Epoch 7/10000
12/12 [==============================] - 4s 378ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 0.5677 - val_ac

KeyboardInterrupt: ignored

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred = np.where(y_pred > 0.5, 1 ,0)

In [ ]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
keras.backend.clear_session()

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 280 ## 사이즈 조정 가능

train_path = dataset_path+'Car_Images_train/'
valid_path = dataset_path+'Car_Images_valid/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
train_datagen = 

valid_datagen = 


SyntaxError: ignored

#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = 

valid_generator = 


#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [ ]:
from tensorflow.keras.applications import VGG16

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
base_model = VGG16(                 )




#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

#### 4) 성능 평가